In [2]:
from textgenrnn import textgenrnn
import pandas
import numpy as np
import rhymes as r
import importlib
importlib.reload(r)
import re

# #word_intro_model = textgenrnn('weights/intro_model_weights.hdf5',
#                                 vocab_path='weights/intro_model_vocab.json',
#                                 config_path='weights/intro_model_config.json')

word_chorus_model = textgenrnn('weights/chorus_model_weights.hdf5',
                                vocab_path='weights/chorus_model_vocab.json',
                                config_path='weights/chorus_model_config.json')

# #word_verse_model = textgenrnn('weights/verse_model_weights.hdf5',
#                                 vocab_path='weights/verse_model_vocab.json',
#                                 config_path='weights/verse_model_config.json')

# #word_bridge_model = textgenrnn('weights/bridge_model_weights.hdf5',
#                                 vocab_path='weights/bridge_model_vocab.json',
#                                 config_path='weights/bridge_model_config.json')

# #word_outro_model = textgenrnn('weights/outro_model_weights.hdf5',
#                                 vocab_path='weights/outro_model_vocab.json',
#                                 config_path='weights/outro_model_config.json')



Using TensorFlow backend.


In [31]:
#uses model to generate a list of lyrics
def gen_lyrics(model):

    lyrics = model.generate(25, temperature=1.2, max_gen_length=1000, return_as_list=True)
    
    return lyrics

#Input: A list of generated lines
#Output: dictionary where the keys are the rhymes and the values are a list of lines that rhyme
def post_process(lines):

    rhyme_dict = dict()

    #Generate of list of last words from each line
    for index, line in enumerate(lines):
        if (len(line) == 0):
            continue
        try:
            wordsInLine = line.split()
            curr_word = ""
            wordsInLine[-1] = wordsInLine[-1].lower()
            #remove puncuation
            for c in wordsInLine[-1]:
                if ord(c) >= 97 and ord(c) <= 122:
                    curr_word += c
            #only add words (remove any empty strings)
            if curr_word != "":
                IT_RHYMES_BABYYY = False
                for key in rhyme_dict.keys():
                    if (r.doTheyRhyme(curr_word, key)):
                        rhyme_dict[key].append(index)
                        IT_RHYMES_BABYYY = True
                if (not IT_RHYMES_BABYYY):
                    rhyme_dict[curr_word] = [index]
        except:
            print (curr_word, 'was not found in the dictionary')
            continue

    return rhyme_dict

#Input dictionary of rhymes with their corresponding line number
#Output: sorts each list of lines in the dictionary
def sort_dict(rhymes_dict):
    
    #loop through each key
    for key in rhymes_dict.keys():
        #get the list of lines associated with the rhyme
        index_list = rhymes_dict[key]
        length = len(index_list)
        if (length < 2):
            continue
        
        #get a list of each distance between the list of lines
        distances = []
        distances.append(index_list[1] - index_list[0])
        for i in range(1, length - 1):
            distances.append(min(index_list[i] - index_list[i - 1], index_list[i + 1] - index_list[i]))

        distances.append(index_list[length - 1] - index_list[length - 2])

        
        #do some cool sorting stuff where you sort distances and the list of lines in parallel
        distances, index_list = zip(*sorted(zip(distances, index_list)))
        distances, index_list = (list(t) for t in zip(*sorted(zip(distances, index_list))))
        print(index_list)
        print(distances)
        
        #add the sorted list back to the dictionary
        rhymes_dict[key] = index_list
        
#Input dictionary of rhymes with their corresponding line number
#Output: returns a dictionary with rhymes that are one away    
def rhymes_one_apart(rhymes_dict):
    
    removeKeys = []
    
    #loop through each key, keep track of ones to remove
    for key in rhymes_dict.keys():
        #get the list of lines associated with the rhyme
        index_list = rhymes_dict[key]
        length = len(index_list)
        
        if (length < 2):
            removeKeys.append(key)
            continue

        new_list = []
        oneApart = False
        
        #loop through list of rhymes and only keep keys that have rhymes one line apart
        for i in range(1, length - 1):
            if (index_list[i + 1] - index_list[i] == 1 or index_list[i] - index_list[i - 1] == 1):
                oneApart = True
                new_list.append(index_list[i])
                
        #Check front and end cases
        if (index_list[1] - index_list[0] == 1):
            new_list.append(index_list[0])
        if (index_list[length - 1] - index_list[length - 2] == 1):
            new_list.append(index_list[length - 1])
        
        #add new list to key or remove the key if there were no rhymes one apart
        if (oneApart):
            print("keep", rhymes_dict[key])
            rhymes_dict[key] = new_list
        else:
            print("remove", key)
            removeKeys.append(key)
            
    for key in removeKeys:
        rhymes_dict.pop(key)
        
                


#Uses dictionary of rhyming lines to generate lyrics with a user's rhyming sequence
def out1(lines, rhyme_dictionary):
    print ('What rhyming scheme would you like to use? (A to Z)')
    rhyme_scheme = input()
    if (not re.fullmatch('[A-Z]+', rhyme_scheme)):
        print ('That is not a valid rhyming scheme.')
        exit()

    # count the required number of lines for each letter/section of the rhyming scheme
    rhyme_counts = dict()
    for char in rhyme_scheme:
        if char in rhyme_counts:
            rhyme_counts[char] += 1
        else:
            rhyme_counts[char] = 1
    print (rhyme_counts)

    # map each letter/section of the rhyming scheme to a rhyme in rhyme_dict
    used_rhymes = set()
    letter_to_rhyme = dict()
    for letter in rhyme_counts.keys():
        for rhyme in rhyme_dictionary.keys():
            # if the current rhyme has enough rhymes for the current letter in the rhyming scheme
            # AND the rhyme hasn't been used, use it
            if (len(rhyme_dictionary[rhyme]) >= rhyme_counts[letter] and rhyme not in used_rhymes):
                letter_to_rhyme[letter] = rhyme_dictionary[rhyme]
                used_rhymes.add(rhyme)
                break

    print (letter_to_rhyme)

    # print the results!
    rhyme_indices = dict()
    for char in rhyme_scheme:
        if (char not in rhyme_indices):
            print (lines[letter_to_rhyme[char][0]])
            rhyme_indices[char] = 1
        else:
            print (lines[letter_to_rhyme[char][rhyme_indices[char]]])
            rhyme_indices[char] += 1

In [32]:
lines = gen_lyrics(word_chorus_model)
lines2 = []
for line in lines:
    split = line.splitlines()
    for l in split:
        lines2.append(l)

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:40<00:00,  1.82s/it]


In [33]:
rhymes_dict = post_process(lines2)
#sort_dict(rhymes_dict)
rhymes_one_apart(rhymes_dict)

niggas was not found in the dictionary
dont was not found in the dictionary
primera was not found in the dictionary
pedir was not found in the dictionary
remove levitating
remove a
remove star
remove side
remove rock
remove toes
keep [33, 34, 122]
remove oh
remove fun
remove leaving
remove that
remove something
remove nothin
remove lovin
remove morning
remove core
remove esta
remove am


In [35]:
out1(lines2, rhymes_dict)

What rhyming scheme would you like to use? (A to Z)
AA
{'A': 2}
{'A': [34, 33]}
but too full too loud
so look in sex sex without


In [16]:
for line in lines:
    print(line)

when you re with me
we are all we need out of salt, tears are running dry
give me you girl.
you're - coconut

ooh
why you acting like that?
(ooh like a fool

ooh
when you walk away from me, where were you down
(ooh)
you don t know how it

no, don t have to be the ordinary of the life of the life the of lawn
holding
the life the of the kind of the of the girl do
what the to life you learned the of the girl of the girl
you gotta have that money
who are you?
i just can t figure out from what it anywhere
without
out wait what it s more than em? (what you?
it okay now, i tell em tell what i now

diamond district in (knew


(
thinking of you high, oh you can t you bout ink,
,?, a t matter what you what you what about you about if you re almost high, oh baby
can t get excited (
i been hoping you guns
comes

so if i could love your
money you and me i know (eh

as i stand here before my woman
i cant far,
and, a, portland
you

i think you re kind of crazy
and not that s good, oh - oh - oh - oh
t

In [34]:
print(rhymes_dict)

{'without': [34, 33]}


In [29]:
print(lines2[78])

in my blood to ball
